
#**Here, my friend, is some simple example of basic SQL query with magic**   

This is a practice of some basic SQL commands on BigQuey with magic in jupyter notebook.

In [0]:
# in colab

from google.colab import auth

auth.authenticate_user()
print('Authenticated')

Authenticated


## Exploring dataset and tables


In [0]:
%%bigquery --project sql-python

SELECT
 * EXCEPT(schema_owner)
FROM
 INFORMATION_SCHEMA.SCHEMATA

In [0]:
%%bigquery --project sql-python

SELECT
  * EXCEPT(is_typed)
FROM
  kaggle_fifa19.INFORMATION_SCHEMA.TABLES

In [0]:
%%bigquery --project sql-python

SELECT
  * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
  kaggle_fifa19.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name="data19_3"

In [0]:
%%bigquery --project sql-python

SELECT
  *
FROM
  kaggle_fifa19.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
  table_name="data19_3"
  AND column_name='Position'

advanced example
https://cloud.google.com/bigquery/docs/information-schema-tables

In [0]:
%%bigquery --project sql-python

SELECT
  * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
  kaggle_fifa19.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name="data19_basic"

In [0]:
%%bigquery --project sql-python

SELECT *
FROM kaggle_fifa19.data19_3
ORDER BY ListID
LIMIT 10 

,ID,Name,Age,Nationality,Club,Preferred_Foot,Position,Jersey_Number,Joined,Contract_Valid_Until,Height_CM_,Weight_KG_,Value_EUR_,Wage_EUR_,Release_Clause_EUR_,ListID
0,158023,L. Messi,31,Argentina,FC Barcelona,Left,RF,10.0,"Jul 1, 2004",2021,170,72,110500000.0,565000000.0,226500000.0,1
1,20801,Cristiano Ronaldo,33,Portugal,Juventus,Right,ST,7.0,"Jul 10, 2018",2022,188,83,77000000.0,405000000.0,127100000.0,2
2,190871,Neymar Jr,26,Brazil,Paris Saint-Germain,Right,LW,10.0,"Aug 3, 2017",2022,175,68,118500000.0,290000000.0,228100000.0,3
3,193080,De Gea,27,Spain,Manchester United,Right,GK,1.0,"Jul 1, 2011",2020,193,76,72000000.0,260000000.0,138600000.0,4
4,192985,K. De Bruyne,27,Belgium,Manchester City,Right,RCM,7.0,"Aug 30, 2015",2023,180,70,102000000.0,355000000.0,196400000.0,5
5,183277,E. Hazard,27,Belgium,Chelsea,Right,LF,10.0,"Jul 1, 2012",2020,173,74,93000000.0,340000000.0,172100000.0,6
6,177003,L. Modrić,32,Croatia,Real Madrid,Right,RCM,10.0,"Aug 1, 2012",2020,173,66,67000000.0,420000000.0,137400000.0,7
7,176580,L. Suárez,31,Uruguay,FC Barcelona,Right,RS,9.0,"Jul 11, 2014",2021,183,86,80000000.0,455000000.0,164000000.0,8
8,155862,Sergio Ramos,32,Spain,Real Madrid,Right,RCB,15.0,"Aug 1, 2005",2020,183,82,51000000.0,380000000.0,104600000.0,9
9,200389,J. Oblak,25,Slovenia,Atlético Madrid,Right,GK,1.0,"Jul 16, 2014",2021,188,87,68000000.0,94000000.0,144500000.0,10


## Creating an empty table

In [0]:
%%bigquery --project sql-python

CREATE TABLE IF NOT EXISTS kaggle_fifa19.data19_bio (
  ID  INT64,
  Height INT64,
  Weight INT64
)
OPTIONS(
   expiration_timestamp=TIMESTAMP "2020-12-01 00:00:00 UTC",
   description="Table for inner join",
   labels=[("org_unit", "development")]
 )



""


## Adding columns by replacing table
No add column DDL in BQ

In [0]:
%%bigquery --project sql-python
CREATE OR REPLACE TABLE kaggle_fifa19.data19_bio (
  ID  INT64,
  Name STRING,
  Age INT64,
  Height INT64,
  Weight INT64
);

ALTER TABLE IF EXISTS kaggle_fifa19.data19_bio
SET OPTIONS(description="Table for inner join")

""


## Inserting and updating data manually

In [0]:
%%bigquery --project sql-python

INSERT kaggle_fifa19.data19_bio (ID, Name, Height, Weight, Age)
VALUES(158023, "L. Messi", 170, 72, 31),
      (20801,	"Cristiano Ronaldo", 188, 83, 33);

UPDATE kaggle_fifa19.data19_bio
SET Name="C. Ronaldo"
WHERE Name like '%Ronaldo%';

SELECT *
FROM kaggle_fifa19.data19_bio

,ID,Name,Age,Height,Weight
0,158023,L. Messi,31,170,72
1,20801,C. Ronaldo,33,188,83


In [0]:
%%bigquery --project sql-python

DELETE kaggle_fifa19.data19_bio
WHERE ID=20801;

SELECT *
FROM kaggle_fifa19.data19_bio


,ID,Name,Age,Height,Weight
0,158023,L. Messi,31,170,72


## Inserting data by copying from another table

In [0]:
%%bigquery --project sql-python

INSERT INTO kaggle_fifa19.data19_bio#(ID, Name, Age, Height, Weight)

SELECT ID, Name, Age, Height_CM_, Weight_KG_
FROM kaggle_fifa19.data19_3
ORDER BY ListID
LIMIT 99 OFFSET 1;

SELECT *
FROM kaggle_fifa19.data19_bio

,ID,Name,Age,Height,Weight
0,158023,L. Messi,31,170,72
1,188545,R. Lewandowski,29,183,80
2,189511,Sergio Busquets,29,188,76
3,202652,R. Sterling,23,170,69
4,193747,Koke,26,175,74
...,...,...,...,...,...
95,162835,S. Handanovič,33,193,92
96,192387,C. Immobile,28,185,85
97,200389,J. Oblak,25,188,87
98,181872,A. Vidal,31,180,75


## Replace existing table by copying data from another table 
and create Rank column from Wage, Vaule, and ID 

In [0]:
%%bigquery --project sql-python
CREATE OR REPLACE TABLE kaggle_fifa19.data19_basic
 OPTIONS(
   description="Top players basic info"
 ) AS
SELECT
  ID, Name, Nationality, Club, Age, ListID
FROM kaggle_fifa19.data19_3
ORDER BY ListID
LIMIT 100;

SELECT *
FROM kaggle_fifa19.data19_basic

,ID,Name,Nationality,Club,Age,ListID
0,158023,L. Messi,Argentina,FC Barcelona,31,1
1,20801,Cristiano Ronaldo,Portugal,Juventus,33,2
2,190871,Neymar Jr,Brazil,Paris Saint-Germain,26,3
3,193080,De Gea,Spain,Manchester United,27,4
4,192985,K. De Bruyne,Belgium,Manchester City,27,5
...,...,...,...,...,...,...
95,183907,J. Boateng,Germany,FC Bayern München,29,96
96,181872,A. Vidal,Chile,FC Barcelona,31,97
97,181458,I. Perišić,Croatia,Inter,29,98
98,180930,E. Džeko,Bosnia Herzegovina,Roma,32,99


## Creating a new table by copying data from another table 

you can **rename** a table by copying it and droping the old one in BQ

In [0]:
%%bigquery --project sql-python
CREATE OR REPLACE TABLE kaggle_fifa19.data19_value   # same as replacing a table
 OPTIONS(
   description="Top players value info"
 ) AS
SELECT ID, Name, Value_EUR_, Wage_EUR_
FROM kaggle_fifa19.data19_3
ORDER BY ListID
LIMIT 100;

SELECT *
FROM kaggle_fifa19.data19_value

,ID,Name,Value_EUR_,Wage_EUR_
0,158023,L. Messi,110500000.0,565000000.0
1,20801,Cristiano Ronaldo,77000000.0,405000000.0
2,190871,Neymar Jr,118500000.0,290000000.0
3,193080,De Gea,72000000.0,260000000.0
4,192985,K. De Bruyne,102000000.0,355000000.0
...,...,...,...,...
95,183907,J. Boateng,30000000.0,115000000.0
96,181872,A. Vidal,26000000.0,205000000.0
97,181458,I. Perišić,37500000.0,110000000.0
98,180930,E. Džeko,30500000.0,115000000.0


## Display columns of all new created/updated tables

In [0]:
%%bigquery --project sql-python

SELECT
  * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
  kaggle_fifa19.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name="data19_basic" or table_name="data19_bio" or table_name="data19_value"



## Join tables
In BQ no primary key nor foreign key needed




In [0]:
%%bigquery --project sql-python
CREATE OR REPLACE TABLE kaggle_fifa19.data19_new
AS
SELECT 
data19_basic.ListID, data19_basic.ID, data19_basic.Name, data19_basic.Nationality, data19_basic.Club, 
data19_bio.Height, data19_bio.Weight, 
data19_value.Value_EUR_,  data19_value.Wage_EUR_
FROM kaggle_fifa19.data19_basic INNER JOIN kaggle_fifa19.data19_bio USING (ID)  INNER JOIN kaggle_fifa19.data19_value USING (ID)
ORDER BY ListID;


In [0]:
%%bigquery --project sql-python df

SELECT * FROM kaggle_fifa19.data19_new ORDER BY ListID

In [0]:
df

,ListID,ID,Name,Nationality,Club,Height,Weight,Value_EUR_,Wage_EUR_
0,1,158023,L. Messi,Argentina,FC Barcelona,170,72,110500000.0,565000000.0
1,2,20801,Cristiano Ronaldo,Portugal,Juventus,188,83,77000000.0,405000000.0
2,3,190871,Neymar Jr,Brazil,Paris Saint-Germain,175,68,118500000.0,290000000.0
3,4,193080,De Gea,Spain,Manchester United,193,76,72000000.0,260000000.0
4,5,192985,K. De Bruyne,Belgium,Manchester City,180,70,102000000.0,355000000.0
...,...,...,...,...,...,...,...,...,...
95,96,183907,J. Boateng,Germany,FC Bayern München,193,90,30000000.0,115000000.0
96,97,181872,A. Vidal,Chile,FC Barcelona,180,75,26000000.0,205000000.0
97,98,181458,I. Perišić,Croatia,Inter,185,80,37500000.0,110000000.0
98,99,180930,E. Džeko,Bosnia Herzegovina,Roma,193,84,30500000.0,115000000.0


## Exporting data 
1. save dataframe on disk
2. use Dataflow https://cloud.google.com/dataflow/docs

## Dropping a table

In [0]:
%%bigquery --project sql-python

DROP TABLE IF EXISTS kaggle_fifa19.data19_1

""


## **Reference**

getting started colab bq https://colab.research.google.com/notebooks/bigquery.ipynb

quick setup colab bq 
https://tech.aaronteoh.com/bigquery-colaboratory-basics/

dataset and table metadata
https://cloud.google.com/bigquery/docs/information-schema-tables

DML syntax https://cloud.google.com/bigquery/docs/reference/standard-sql/dml-syntax

standard SQL query syntax https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax

w3schools SQL tutorial https://www.w3schools.com/sql/default.asp


